In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime
import logging

In [2]:
csv_file_path = 'employees2.csv'
df = pd.read_csv("C:/Users/varsh/Downloads/employees2.csv")
print("Raw data loaded:")
print(df.head())
print(df.columns.tolist())
logging.info("CSV loaded successfully.")

Raw data loaded:
   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  
['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [3]:
df.fillna({
'EMAIL': 'not_provided@example.com',
'PHONE_NUMBER': '0000000000',
'HIRE_DATE': '01-Jan-00',
'SALARY': 0
}, inplace=True)

In [4]:
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

In [5]:
print(df.columns.tolist())

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [6]:
df['hire_date'] = pd.to_datetime(df['hire_date'], format='%d-%b-%y',
errors='coerce')
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int)

In [7]:
logging.info("Data cleaning completed.")

In [8]:
mydb = mysql.connector.connect(
host="localhost",
user="root",
password="Varshini@07",
database="employee" # <-- change this
)
cursor = mydb.cursor()

In [9]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS salary_2 (
empid INT,
firstname VARCHAR(50),
lastname VARCHAR(50),
email VARCHAR(100),
phone VARCHAR(20),
hire_date DATE,
job_id VARCHAR(20),
salary INT
)
""")

In [10]:
for index, row in df.iterrows():
    sql = """
INSERT INTO salary_2 (empid, firstname, lastname, email, phone,
hire_date, job_id, salary)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
firstname=VALUES(firstname),
lastname=VALUES(lastname),
email=VALUES(email),
phone=VALUES(phone),
hire_date=VALUES(hire_date),
job_id=VALUES(job_id),
salary=VALUES(salary)
"""
values = (
int(row['employee_id']),
row['first_name'],
row['last_name'],
row['email'],
row['phone_number'],
row['hire_date'].date(),
row['job_id'],
int(row['salary'])
)
cursor.execute(sql, values)

In [11]:
mydb.commit()
cursor.close()
mydb.close()
logging.error("Something went wrong", exc_info=True)
print("ETL process completed successfully.")

ERROR:root:Something went wrong
NoneType: None


ETL process completed successfully.


In [12]:
import os
print(os.getcwd())

C:\Users\varsh


In [24]:
!jupyter nbconvert --to script ETL.ipynb

This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr

[NbConvertApp] WARNING | pattern 'ETL.ipynb' matched no files
